In [2]:
import os
import megengine as mge
import megengine.functional as F
import argparse
import numpy as np
import cv2
from nets import Model
from matplotlib import pyplot as plt
from PIL import Image
import glob

In [1]:
def load_model(model_path):
    print("Loading model:", os.path.abspath(model_path))
    pretrained_dict = mge.load(model_path)
    model = Model(max_disp=256, mixed_precision=False, test_mode=True)
    model.load_state_dict(pretrained_dict["state_dict"], strict=True)
    model.eval()
    return model

def inference(left, right, model, n_iter=20):
    print("Model Forwarding...")
    imgL = left.transpose(2, 0, 1)
    imgR = right.transpose(2, 0, 1)
    imgL = np.ascontiguousarray(imgL[None, :, :, :])
    imgR = np.ascontiguousarray(imgR[None, :, :, :])
    imgL = mge.tensor(imgL).astype("float32")
    imgR = mge.tensor(imgR).astype("float32")
    imgL_dw2 = F.nn.interpolate(
        imgL,
        size=(imgL.shape[2] // 2, imgL.shape[3] // 2),
        mode="bilinear",
        align_corners=True,
    )
    imgR_dw2 = F.nn.interpolate(
        imgR,
        size=(imgL.shape[2] // 2, imgL.shape[3] // 2),
        mode="bilinear",
        align_corners=True,
    )
    pred_flow_dw2 = model(imgL_dw2, imgR_dw2, iters=n_iter, flow_init=None)
    pred_flow = model(imgL, imgR, iters=n_iter, flow_init=pred_flow_dw2)
    pred_disp = F.squeeze(pred_flow[:, 0, :, :]).numpy()
    return pred_disp

#loading the model
model_func = load_model("crestereo_eth3d.mge")

#loading the rectification parametrs
cv_file = cv2.FileStorage()
cv_file.open('../stereoMap.xml', cv2.FileStorage_READ)
stereoMapL_x = cv_file.getNode('stereoMapL_x').mat()
stereoMapL_y = cv_file.getNode('stereoMapL_y').mat()
stereoMapR_x = cv_file.getNode('stereoMapR_x').mat()
stereoMapR_y = cv_file.getNode('stereoMapR_y').mat()

#creating an avi file for the output
out = cv2.VideoWriter('disp_body_test.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (1280, 720))

#loading the left and right videos
vidcap_left = cv2.VideoCapture('../body_test_left.mp4')
vidcap_right = cv2.VideoCapture('../body_test_right.mp4')
success_left,left = vidcap_left.read()
success_right,right = vidcap_right.read()
count = 0

while success_left and success_right:
    
    left = cv2.remap(left, stereoMapL_x, stereoMapL_y, cv2.INTER_LINEAR)
    right= cv2.remap(right, stereoMapR_x, stereoMapR_y, cv2.INTER_LINEAR)
    
    in_h, in_w = left.shape[:2]
    eval_h, eval_w = [int(e) for e in (1024,1536)]
    left_img = cv2.resize(left, (eval_w, eval_h), interpolation=cv2.INTER_LINEAR)
    right_img = cv2.resize(right, (eval_w, eval_h), interpolation=cv2.INTER_LINEAR)
    pred = inference(left_img, right_img, model_func, n_iter=20)
    t = float(in_w) / float(eval_w)
    disp = cv2.resize(pred, (in_w, in_h), interpolation=cv2.INTER_LINEAR) * t
    disp_vis = (disp - disp.min()) / (disp.max() - disp.min()) * 255.0
    disp_vis = disp_vis.astype("uint8")
    disp_vis = cv2.applyColorMap(disp_vis, cv2.COLORMAP_INFERNO)
    out.write(disp_vis)
    success_left,left = vidcap_left.read()
    success_right,right = vidcap_right.read()
    count += 1
out.release()

In [23]:
# out = cv2.VideoWriter('Rotated_face_test_left.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (720, 1280))
# vidcap= cv2.VideoCapture('../face_test_left.mp4')
# success,img = vidcap.read()
# while success:
#     rotated=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
#     out.write(rotated)
#     success,img = vidcap.read()
# out.release()

# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
# ffmpeg_extract_subclip("Rotated_face_test_left.avi", 6, 32, targetname="Rotated_face_test_left.mp4")